Purpose: Get a random set of books with GoodReads reviews and their covers

In [1]:
import pandas as pd
import numpy as np 

import requests
from bs4 import BeautifulSoup 

import time

### 1. Get set of books

We'll use a a random subset of books from this Kaggle dataset: https://www.kaggle.com/jealousleopard/goodreadsbooks?select=books.csv

(note a few records had to be cleaned in the csv for inconsistent formatting) 

In [2]:
books = pd.read_csv('books.csv')
len(books)

11127

In [3]:
# only choose books with at least 50 reviews - arbitrary threshold to get reasonably accurate rating
books = books[books['ratings_count']>=50]
len(books)

9239

In [4]:
# choose a more manageable sample
sample = books.sample(1000, random_state=11)

### 2. Download images

GoodReads' API doesn't have images for the most part, so we have to do this by web

In [11]:
# download pictures

# pick up from break
ix = 897

for i in sample['bookID'][ix:]: 
    url = f'https://www.goodreads.com/book/show/{i}'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        img_url = soup.find(id='coverImage').get('src')
    except AttributeError: 
        time.sleep(600)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        img_url = soup.find(id='coverImage').get('src')
    
    r2 = requests.get(img_url)

    file = open(f'imgs/{i}.jpg', "wb")
    file.write(r2.content)
    file.close()
    
    # limit rate
    time.sleep(10)